In [ ]:
library(GenomicRanges)
library(dplyr)
library(data.table)
library(ggplot2)
library(viridis)
library(readxl)
library(pheatmap)
library(RColorBrewer)
library(tidyr)
library(gridExtra)
library(corrplot)
library(tibble)
library(ggrepel)
library(ggpubr)
library(DEFormats)
library(BiocParallel)
register(MulticoreParam(4))
library(chromVAR)
library(GenomicFeatures)
library(BSgenome.Hsapiens.UCSC.hg38)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(org.Hs.eg.db)
library(pheatmap)
library(RColorBrewer)
library(tidyr)
library(gridExtra)
library(ggrepel)
library(limma)
library(ggdendro)
library(ggfortify)
library(ggpubr)
library(magrittr)
library(Seurat)

In [2]:
histList = c("H3K27me3", "H3K4me2")
topN = 20
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
load(file = paste0(outPath, "/RData/PatientData_peakOverlap_withDuplicates_seacrOnly_top0_", topN, ".RData"))
load(file = paste0(outPath, "/RData/PatientData_IDR_master_peak_list_chromVar_count_histList_noChrXYM_top0_", topN, ".RData"))
load(file = paste0(outPath, "/RData/PatientData_countMat_designInfo_histList_noChrXYM_top0_", topN, ".RData"))

In [ ]:
countMat$H3K27me3 %>% head
countMat$H3K4me2 %>% head

patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
info = patientRaw %>% data.frame %>% dplyr::select(Xnumber, candr_day, age, sex, bestresp) 

countMat$H3K27me3 = countMat$H3K27me3[, patientRaw$Xnumber]
countMat$H3K4me2 = countMat$H3K4me2[, patientRaw$Xnumber]

infoData = list(H3K27me3 = data.frame(Xnumber = colnames(countMat[[histList[1]]])) %>% left_join(., info, by = "Xnumber") %>% mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))), 
                H3K4me2 = data.frame(Xnumber = colnames(countMat[[histList[2]]])) %>% left_join(., info, by = "Xnumber") %>% mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))))

colnames(countMat$H3K27me3) = paste0(infoData$H3K27me3$Xnumber, "_", infoData$H3K27me3$bestresp)
colnames(countMat$H3K4me2) = paste0(infoData$H3K4me2$Xnumber, "_", infoData$H3K4me2$bestresp)

In [ ]:
responseList = c("CR", "PR", "SD", "PD")
hdList <- infoData$H3K4me2$Xnumber
histList <- c("H3K27me3", "H3K4me2")
target = infoData$H3K4me2 %>% dplyr::select(Xnumber, bestresp)

# target = c()
# for(cell in cellList){
#   for(expr in exprList){
#     hdL = checkHD(expr)
#     for(hd in hdL){
#       if( !((cell == "EMRA") & (expr == "Input") && (hd == "HD2"))){
#         target = rbind(target, data.frame(cell = cell, expr = expr, humanDonor = hd))
#       }

#     }
#   }
# }
target$bestresp = factor(target$bestresp, levels = responseList)
target  = target %>% dplyr::filter(bestresp %in% c("CR", "PD"))
target

## Experimental design
treat <- factor(target$bestresp)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(CR_PD = CR - PD, levels = design)


In [64]:

## read in genes location
gtf_filter_df = rtracklayer::import('/fh/fast/gottardo_r/yezheng_working/SupplementaryData/hg38/GENCODE/gencode.v21.annotation.gtf') %>% 
    as.data.frame %>% 
    dplyr::filter(type == "gene", !(seqnames %in% c("chrM", "chrX", "chrY")))
gene_gr = GRanges(seqnames = gtf_filter_df$seqnames, ranges = IRanges(start = gtf_filter_df$start, end = gtf_filter_df$end), strand = gtf_filter_df$strand, gene_id = gtf_filter_df$gene_id, gene_name = gtf_filter_df$gene_name, gene_type = gtf_filter_df$gene_type)



In [72]:
min_dist_peak_to_gene = 5000
results = vector("list", length(histList))
results_fullInfo = vector("list", length(histList))
voomDDS = vector("list", length(histList))
for(hist in histList){
  ## Filter and delete low expressed genes
    selectR = which(rowSums(countMat[[hist]]) > 200) ## remove low count genes
    selectC = which(colnames(countMat[[hist]]) %>% stringr::str_detect("CR") | colnames(countMat[[hist]]) %>% stringr::str_detect("PD"))

    dataS = countMat[[hist]][selectR, selectC]
    peakRegionList = mPeak[[hist]][selectR] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end)) %$% region
    rownames(dataS) = peakRegionList
    voomDDS[[hist]] = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)
    ## option 1 using voomDDS option2 using normDDS as normalized input.
    inputDDS = voomDDS[[hist]]
    ## corfit$consensus
    fit = lmFit(inputDDS, design)
    fitContrast = contrasts.fit(fit, contrast)
    fitBayes = eBayes(fitContrast, robust = TRUE)


    results[[hist]] = list()
    results_fullInfo[[hist]] = list()
    for(i in 1:ncol(contrast)){
      ## Results
      res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = i, number = nrow(inputDDS), sort = 'P')

      res = data.table(PeakRegion = rownames(res), res)
      res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                              ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
      results[[hist]][[i]] = left_join(res, dataS %>% data.frame %>% dplyr::mutate(PeakRegion = rownames(dataS)), by = "PeakRegion")

      ## overlap with genes
      results_peakRegion = results[[hist]][[i]] %$% PeakRegion
      results_chrom = gsub(":.*", "", results_peakRegion)
      results_start = gsub(".*:", "", results_peakRegion) %>% gsub("-.*", "", .) %>% as.numeric
      results_end = gsub(".*:", "", results_peakRegion) %>% gsub(".*-", "", .) %>% as.numeric
      results_peakRegion_gr = GRanges(seqnames = results_chrom, IRanges(start = results_start, end = results_end), strand = "*")
      
      dp_match_gene = distanceToNearest(results_peakRegion_gr, gene_gr, select = "all") ## get the nearest gene for each peak
      dp_match_gene = dp_match_gene[which(data.frame(dp_match_gene)$distance <= min_dist_peak_to_gene)]

      results_fullInfo[[hist]][[i]] = cbind(
                results[[hist]][[i]][dp_match_gene@from, ], 
                gene_gr[dp_match_gene@to] %>% data.frame %>% dplyr::select(gene_name, gene_id, gene_type)
      )
      results_fullInfo[[hist]][[i]]$gene_region = paste0(data.frame(gene_gr[dp_match_gene@to])$seqnames, ":", data.frame(gene_gr[dp_match_gene@to])$start, "-", data.frame(gene_gr[dp_match_gene@to])$end)
      results_fullInfo[[hist]][[i]]$peak_distance_to_gene = data.frame(dp_match_gene)$distance
      ## Output
      write.table(results[[hist]][[i]], file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_CR_VS_PD_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")
      write.table(results_fullInfo[[hist]][[i]], file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_CR_VS_PD_adj0.1_logFC2_link_nearest_genes.csv'), quote = FALSE, row.names = FALSE, sep = ",")
    }
    names(results[[hist]]) <- colnames(contrast)
    names(results_fullInfo[[hist]]) <- colnames(contrast)


}
save(results, results_fullInfo, voomDDS, file = paste0(outPath, "/RData/results_histList_noChrXYM_paitent_CR_vs_PD.RData"))




In [74]:
hist = "H3K27me3"
results_fullInfo[[hist]][["CR_PD"]] %>% arrange(gene_name) %>% group_by(gene_name) %>% top_n(n = 1, wt = -adj.P.Val) %>%
write.table(., file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_CR_VS_PD_adj0.1_logFC2_link_uniqueGene_mostSignificantPeak.csv'), quote = FALSE, row.names = FALSE, sep = ",")


hist = "H3K4me2"
results_fullInfo[[hist]][["CR_PD"]] %>% arrange(gene_name) %>% group_by(gene_name) %>% top_n(n = 1, wt = -adj.P.Val) %>%
write.table(., file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_CR_VS_PD_adj0.1_logFC2_link_uniqueGene_mostSignificantPeak.csv'), quote = FALSE, row.names = FALSE, sep = ",")


In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
selectR = which(rowSums(countMat[[hist]]) > 200) ## remove low count genes
selectC = which(colnames(countMat[[hist]]) %>% stringr::str_detect("CR") | colnames(countMat[[hist]]) %>% stringr::str_detect("PD"))

tmp = countMat[[hist]][selectR, selectC]

k27_obj = CreateSeuratObject(counts = countMat[[hist]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k27_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

hist = "H3K4me2"
selectR = which(rowSums(countMat[[hist]]) > 200) ## remove low count genes
selectC = which(colnames(countMat[[hist]]) %>% stringr::str_detect("CR") | colnames(countMat[[hist]]) %>% stringr::str_detect("PD"))
k4_obj = CreateSeuratObject(counts = countMat[[hist]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k4_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")

cr_obj = CreateSeuratObject(counts = rbind(tmp, countMat[[hist]][selectR, selectC]), project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR) + nrow(tmp)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(cr_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
rremove("legend.title")
dev.off()
pdf("CART_CUTRUN_Project/results/paper_figure/patient-related/Patient_CR_PD_PCA_on_all_peaks.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05)
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD")) 

tmp = results[[hist]][["CR_PD"]][selectR, selectC]
rownames(tmp) = paste0(rownames(tmp), "_H3K27me3")

k27_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k27_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

hist = "H3K4me2"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05) 
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD"))
k4_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k4_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")


cr_obj = CreateSeuratObject(counts = rbind(tmp, results[[hist]][["CR_PD"]][selectR, selectC]), project = "cart", min.cells = 0, min.features = 50) %>% 
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR) + nrow(tmp)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(cr_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
rremove("legend.title")
dev.off()
pdf("CART_CUTRUN_Project/results/paper_figure/patient-related/Patient_CR_PD_PCA_on_differentialPeak_pvalue0.05.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [ ]:
## use TSS region for differential detection
## Get the TSS location and +/-1kb region around TSS
gtf <- rtracklayer::import('/fh/fast/gottardo_r/yezheng_working/SupplementaryData/hg38/GENCODE/gencode.v21.annotation.gtf')
gtf_df=as.data.frame(gtf)


## get the TSS location and extend by 1kb up and downstream
tss_df = rbind(
    gtf_df %>% data.frame %>% dplyr::filter(type == "gene", strand == "+") %>% dplyr::select(seqnames, TSS = start, strand, source, gene_id, gene_name, gene_name, gene_type) ,
    gtf_df %>% data.frame %>% dplyr::filter(type == "gene", strand == "-") %>% dplyr::select(seqnames, TSS = end, strand, source, gene_id, gene_name, gene_name, gene_type) 
)
tss_filter_df = tss_df %>% dplyr::filter(!(seqnames %in% c("chrM", "chrX", "chrY")))

tss_gr = GRanges(seqnames = tss_filter_df$seqnames, ranges = IRanges(start = tss_filter_df$TSS - 1000, end = tss_filter_df$TSS + 1000), strand = tss_filter_df$strand, gene_id = tss_filter_df$gene_id, gene_name = tss_filter_df$gene_name, gene_type = tss_filter_df$gene_type)
tss_gr 


In [3]:
## overlap with bam file to get count -- !!! Run it in terminal directly
# library(doMC)
# coreN <- 6
# registerDoMC(cores = coreN)

outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
histList = c("H3K27me3", "H3K4me2")
# bamSelect = vector("list", length(histList))
# patient_ordered = vector("list", length(histList))
# fragment_tss_counts = vector("list", length(histList))
# ## dataset 1
# inPath = "/shared/ngs/illumina/sfiorenz/201014_D00300_1082_AHHYJJBCX3/analysis/"
# index = grep("IP", list.files(inPath), fixed = TRUE)[!grep("IP", list.files(inPath), fixed = TRUE) %in% grep("IgG", list.files(inPath), fixed = TRUE)]
# sampleL = list.files(inPath)[index]
# sampleLtmp = sampleL %>% strsplit("_") %>% unlist
# patientL = sampleLtmp[seq(4, length(sampleLtmp), 7)]
# histL = sampleLtmp[seq(6, length(sampleLtmp), 7)]

# for(i in 1:length(sampleL)){
#   sample = sampleL[i]
#   hist = histL[i]
#   pt = patientL[i]
#   bamFile = paste0(inPath, sample, "/bowtie2_align.bam")
#   if(file.exists(bamFile)){
#     bamSelect[[hist]] = c(bamSelect[[hist]], bamFile)
#     patient_ordered[[hist]] = c(patient_ordered[[hist]], pt)
#   }else{
#     print(paste0(bamFile, " does not exist!"))
#   }
# }

# ## dataset 2

# inPath = "/shared/ngs/illumina/tphi/200924_D00300_1063_AHHLGCBCX3/analysis/"
# index = grep("IP", list.files(inPath), fixed = TRUE)[!grep("IP", list.files(inPath), fixed = TRUE) %in% grep("IgG", list.files(inPath), fixed = TRUE)]
# sampleL = list.files(inPath)[index]
# sampleLtmp = sampleL %>% strsplit("_") %>% unlist
# patientL = sampleLtmp[seq(3, length(sampleLtmp), 6)]
# histL = sampleLtmp[seq(5, length(sampleLtmp), 6)]

# for(i in 1:length(sampleL)){
#   sample = sampleL[i]
#   hist = histL[i]
#   pt = patientL[i]
#   bamFile = paste0(inPath, sample, "/bowtie2_align.bam")
#   if(file.exists(bamFile)){
#     bamSelect[[hist]] = c(bamSelect[[hist]], bamFile)
#     patient_ordered[[hist]] = c(patient_ordered[[hist]], pt)
#   }else{
#     print(paste0(bamFile, " does not exist!"))
#   }
# }

# ## read in the reads
# for(hist in histList){
#   fragment_tss_counts[[hist]] <- mclapply(as.list(bamSelect[[hist]]), chromVAR::getCounts, tss_gr, paired = TRUE, by_rg = FALSE, format = "bam", mc.cores = coreN)
# }

# saveRDS(fragment_tss_counts, file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatients_noChrXYM.rds"))
# saveRDS(list(bamSelect, patient_ordered), file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatients_noChrXYM_file_pt_info.rds"))
fragment_tss_counts = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatients_noChrXYM.rds"))
tmp = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatients_noChrXYM_file_pt_info.rds"))
bamSelect = tmp[[1]]
patient_ordered = tmp[[2]]


In [4]:
# library(chromVAR)
# library(SummarizedExperiment)
# library(Matrix)

# ## count matrix column name
# countMat = vector("list", length(histList))
# normMat = vector("list", length(histList))
# seqDepth = vector("list", length(histList))
# designInfo = c()
# for(hist in histList){

#   countMat[[hist]] <- matrix(NA, length(tss_gr), length(bamSelect[[hist]]))
#   normMat[[hist]] <- matrix(NA, length(tss_gr), length(bamSelect[[hist]]))
#   colnames(countMat[[hist]]) <- patient_ordered[[hist]]
#   colnames(normMat[[hist]]) <- patient_ordered[[hist]]
#   seqDepth[[hist]] <- NULL
#   for (k in 1:length(bamSelect[[hist]])){
#     countMat[[hist]][, k] <- counts(fragment_tss_counts[[hist]][[k]])[,1]
#     seqDepth[[hist]][k] <- fragment_tss_counts[[hist]][[k]]@colData[1,1]
#     normMat[[hist]][, k] <- countMat[[hist]][, k]/seqDepth[[hist]][k] * 14847059 ## divide by the seqDepth and multiply by the largest seqDepth
#   }
#   designInfo <- data.frame(exps = patient_ordered[[hist]], depth = seqDepth[[hist]], hist = hist) %>% rbind(designInfo, .)
# }
# rownames(countMat$H3K4me2) = data.frame(tss_gr)$gene_name
# rownames(countMat$H3K27me3) = data.frame(tss_gr)$gene_name
# rownames(normMat$H3K4me2) = data.frame(tss_gr)$gene_name
# rownames(normMat$H3K27me3) = data.frame(tss_gr)$gene_name

# ## read in patient info
# patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
# info = patientRaw %>% data.frame %>% dplyr::select(Xnumber, candr_day, age, sex, bestresp) 

# countMat$H3K27me3 = countMat$H3K27me3[, patientRaw$Xnumber]
# countMat$H3K4me2 = countMat$H3K4me2[, patientRaw$Xnumber]

# infoData = list(H3K27me3 = data.frame(Xnumber = colnames(countMat[[histList[1]]])) %>% left_join(., info, by = "Xnumber") %>% mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))), 
#                 H3K4me2 = data.frame(Xnumber = colnames(countMat[[histList[2]]])) %>% left_join(., info, by = "Xnumber") %>% mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))))

# colnames(countMat$H3K27me3) = paste0(infoData$H3K27me3$Xnumber, "_", infoData$H3K27me3$bestresp)
# colnames(countMat$H3K4me2) = paste0(infoData$H3K4me2$Xnumber, "_", infoData$H3K4me2$bestresp)


# saveRDS(countMat, file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_countMat.rds"))
# saveRDS(normMat, file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_normMat.rds"))
# saveRDS(designInfo, file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_designInfo.rds"))

countMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_NHLpatient_noChrXYM_countMat.rds"))
normMat = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_histList_hd1-7_noChrXYM_normMat.rds"))
designInfo = readRDS(file = paste0(outPath, "/RData/TSS_neighbors_chromVar_count_histList_hd1-7_noChrXYM_designInfo.rds"))


In [19]:
responseList = c("CR", "PR", "SD", "PD")
hdList <- infoData$H3K4me2$Xnumber
histList <- c("H3K27me3", "H3K4me2")
target = infoData$H3K4me2 %>% dplyr::select(Xnumber, bestresp)

target$bestresp = factor(target$bestresp, levels = responseList)
target  = target %>% dplyr::filter(bestresp %in% c("CR", "PD"))

## Experimental design
treat <- factor(target$bestresp)
design <- model.matrix(~0 + treat)
colnames(design) <- levels(treat)
contrast <- makeContrasts(CR_PD = CR - PD, levels = design)

In [5]:
# min_dist_peak_to_gene = 5000
# results = vector("list", length(histList))
# results_fullInfo = vector("list", length(histList))
# voomDDS = vector("list", length(histList))
# for(hist in histList){
#   ## Filter and delete low expressed genes
#     selectR = which(rowSums(countMat[[hist]]) > 50) ## remove low count genes
#     selectC = which(colnames(countMat[[hist]]) %>% stringr::str_detect("CR") | colnames(countMat[[hist]]) %>% stringr::str_detect("PD"))

#     dataS = countMat[[hist]][selectR, selectC]
#     peakRegionList = tss_gr[selectR] %>% data.frame %>% mutate(region = paste0(seqnames, ":", start, "-", end, "(", gene_name, ")")) %$% region
#     rownames(dataS) = peakRegionList
#     voomDDS[[hist]] = voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)
#     ## option 1 using voomDDS option2 using normDDS as normalized input.
#     inputDDS = voomDDS[[hist]]
#     ## corfit$consensus
#     fit = lmFit(inputDDS, design)
#     fitContrast = contrasts.fit(fit, contrast)
#     fitBayes = eBayes(fitContrast, robust = TRUE)


#     results[[hist]] = list()
#     results_fullInfo[[hist]] = list()
#     for(i in 1:ncol(contrast)){
#       ## Results
#       res = topTable(fit = fitBayes, adjust.method = 'fdr', coef = i, number = nrow(inputDDS), sort = 'P') %>% data.table

#       # res = data.table(PeakRegion = rownames(res), res)
#       res[, Significance := ifelse((adj.P.Val <= 0.1 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
#                               ifelse((adj.P.Val <= 0.1 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
#       results[[hist]][[i]] = left_join(res, dataS %>% data.frame %>% dplyr::mutate(ID = rownames(dataS)), by = "ID")

#       ## overlap with genes
#       # results_peakRegion = results[[hist]][[i]] %$% PeakRegion
#       # results_chrom = gsub(":.*", "", results_peakRegion)
#       # results_start = gsub(".*:", "", results_peakRegion) %>% gsub("-.*", "", .) %>% as.numeric
#       # results_end = gsub(".*:", "", results_peakRegion) %>% gsub(".*-", "", .) %>% as.numeric
#       # results_peakRegion_gr = GRanges(seqnames = results_chrom, IRanges(start = results_start, end = results_end), strand = "*")
      
#       # dp_match_gene = distanceToNearest(results_peakRegion_gr, gene_gr, select = "all") ## get the nearest gene for each peak
#       # dp_match_gene = dp_match_gene[which(data.frame(dp_match_gene)$distance <= min_dist_peak_to_gene)]

#       # results_fullInfo[[hist]][[i]] = cbind(
#       #           results[[hist]][[i]][dp_match_gene@from, ], 
#       #           gene_gr[dp_match_gene@to] %>% data.frame %>% dplyr::select(gene_name, gene_id, gene_type)
#       # )
#       # results_fullInfo[[hist]][[i]]$gene_region = paste0(data.frame(gene_gr[dp_match_gene@to])$seqnames, ":", data.frame(gene_gr[dp_match_gene@to])$start, "-", data.frame(gene_gr[dp_match_gene@to])$end)
#       # results_fullInfo[[hist]][[i]]$peak_distance_to_gene = data.frame(dp_match_gene)$distance
#       # ## Output
#       write.table(results[[hist]][[i]], file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_TSSregions_CR_VS_PD_adj0.1_logFC2.csv'), quote = FALSE, row.names = FALSE, sep = ",")
#       # write.table(results_fullInfo[[hist]][[i]], file = paste0(outPath, '/CUTRUN_limma_tables/DE_', hist, '_patient_TSSregions_CR_VS_PD_adj0.1_logFC2_link_nearest_genes.csv'), quote = FALSE, row.names = FALSE, sep = ",")
#     }
#     names(results[[hist]]) <- colnames(contrast)
#     # names(results_fullInfo[[hist]]) <- colnames(contrast)


# }
# save(results, voomDDS, file = paste0(outPath, "/RData/results_histList_noChrXYM_paitent_TSSregions_CR_vs_PD.RData"))
load(file = paste0(outPath, "/RData/results_histList_noChrXYM_paitent_TSSregions_CR_vs_PD.RData"))


In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05)
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD")) 

tmp = results[[hist]][["CR_PD"]][selectR, selectC]
rownames(tmp) = paste0(rownames(tmp), "_H3K27me3")
results[[hist]][["CR_PD"]][selectR, selectC] %>% head

# k27_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
# NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
# FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
# ScaleData()

# k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
# k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k27_obj)), levels = c("CR", "PD"))) %>% 
# ggplot(aes(x = PC_1, y = PC_2, color = response)) +
# geom_point() +
# theme_bw(base_size = 20) +
# scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
# xlab("PC 1") +
# ylab("PC 2") +
# ggtitle("H3K27me3") +
# rremove("legend.title")

In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05)
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD")) 

tmp = results[[hist]][["CR_PD"]][selectR, selectC]
rownames(tmp) = paste0(rownames(tmp), "_H3K27me3")

k27_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k27_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
rremove("legend.title")

hist = "H3K4me2"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05) 
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD"))
k4_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(k4_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
rremove("legend.title")


cr_obj = CreateSeuratObject(counts = rbind(tmp, results[[hist]][["CR_PD"]][selectR, selectC]), project = "cart", min.cells = 0, min.features = 50) %>% 
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR) + nrow(tmp)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(response = factor(gsub(".*_", "", colnames(cr_obj)), levels = c("CR", "PD"))) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = response)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
rremove("legend.title")

pdf("CART_CUTRUN_Project/results/paper_figure/patient-related/Patient_CR_PD_PCA_on_TSSregions_differential_pvalue0.05.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [ ]:
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
info = patientRaw %>% data.frame %>% dplyr::select(Xnumber, candr_day, age, sex, bestresp, spd, ldh) 
info$spd[which(info$spd == "Not evaluable")] = NA
infoData = list(H3K27me3 = data.frame(Xnumber = colnames(countMat[[histList[1]]]) %>% gsub("_.*", "", .)) %>% left_join(., info, by = "Xnumber"), #mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))), 
                H3K4me2 = data.frame(Xnumber = colnames(countMat[[histList[2]]]) %>% gsub("_.*", "", .)) %>% left_join(., info, by = "Xnumber")) #mutate(candr_day = factor(candr_day), age = factor(age), sex = factor(sex), bestresp = factor(bestresp, levels = c("CR", "PR", "SD", "PD"))))
infoData

In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05)
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD")) 

tmp = results[[hist]][["CR_PD"]][selectR, selectC]
rownames(tmp) = paste0(rownames(tmp), "_H3K27me3")

k27_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(SPD = infoData$H3K27me3$spd[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = SPD)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))

hist = "H3K4me2"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05) 
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD"))
k4_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(SPD = infoData$H3K4me2$spd[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = SPD)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))


cr_obj = CreateSeuratObject(counts = rbind(tmp, results[[hist]][["CR_PD"]][selectR, selectC]), project = "cart", min.cells = 0, min.features = 50) %>% 
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR) + nrow(tmp)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(SPD = infoData$H3K4me2$spd[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = SPD)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "magma") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))

pdf("CART_CUTRUN_Project/results/paper_figure/patient-related/Patient_CR_PD_PCA_on_TSSregions_differential_pvalue0.05_spd.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [ ]:
options(repr.plot.width=22, repr.plot.height=10)
hist = "H3K27me3"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05)
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD")) 

tmp = results[[hist]][["CR_PD"]][selectR, selectC]
rownames(tmp) = paste0(rownames(tmp), "_H3K27me3")

k27_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k27_obj = RunPCA(k27_obj, features = VariableFeatures(object = k27_obj), npcs = 15)
k27_plot = Embeddings(k27_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(LDH = infoData$H3K27me3$ldh[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = LDH)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "viridis") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))

hist = "H3K4me2"
results[[hist]][["CR_PD"]] =  data.frame(results[[hist]][["CR_PD"]])

selectR = which(results[[hist]][["CR_PD"]]$P.Value <= 0.05) 
selectC = which(colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("CR") | colnames(results[[hist]][["CR_PD"]]) %>% stringr::str_detect("PD"))
k4_obj = CreateSeuratObject(counts = results[[hist]][["CR_PD"]][selectR, selectC], project = "cart", min.cells = 0, min.features = 50) %>%  
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR)) %>% 
ScaleData()
k4_obj = RunPCA(k4_obj, features = VariableFeatures(object = k4_obj), npcs = 15)

k4_plot = Embeddings(k4_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(LDH = infoData$H3K4me2$ldh[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = LDH)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "viridis") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K4me2") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))


cr_obj = CreateSeuratObject(counts = rbind(tmp, results[[hist]][["CR_PD"]][selectR, selectC]), project = "cart", min.cells = 0, min.features = 50) %>% 
NormalizeData(., normalization.method = "LogNormalize", scale.factor = 1000000) %>% 
FindVariableFeatures(., selection.method = "vst", nfeatures = length(selectR) + nrow(tmp)) %>% 
ScaleData()
cr_obj = RunPCA(cr_obj, features = VariableFeatures(object = cr_obj), npcs = 15)
cr_plot = Embeddings(cr_obj, reduction = "pca")[, 1:2] %>% data.frame %>% mutate(LDH = infoData$H3K4me2$ldh[selectC] %>% as.numeric) %>% 
ggplot(aes(x = PC_1, y = PC_2, color = LDH)) +
geom_point() +
theme_bw(base_size = 20) +
scale_color_viridis(discrete = FALSE, option = "viridis") +
xlab("PC 1") +
ylab("PC 2") +
ggtitle("H3K27me3 + H3K4me2") +
theme(legend.position = "top", legend.key.width = unit(3, "cm"))

pdf("CART_CUTRUN_Project/results/paper_figure/patient-related/Patient_CR_PD_PCA_on_TSSregions_differential_pvalue0.05_ldh.pdf", width = 15, height = 6)
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)
dev.off()
ggarrange(k27_plot, k4_plot, cr_plot, common.legend = TRUE, ncol = 3, nrow = 1)


In [49]:
## rerun peak calling
options(ChIPseeker.ignore_1st_exon = TRUE)
options(ChIPseeker.ignore_1st_intron = TRUE)
options(ChIPseeker.ignore_downstream = TRUE)
options(ChIPseeker.ignore_promoter_subcategory = TRUE)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
annodb <- "org.Hs.eg.db"

convertGR = function(peak){
    peak.gr = GRanges(seqname = peak$V1, IRanges(peak$V2, peak$V3), strand = "*")
    return(peak.gr)
}

convertGR_extend = function(peak){
    peak.gr = GRanges(seqname = peak$V1, IRanges(peak$V2 - 500, peak$V3 + 500), strand = "*") %>% reduce
    start(peak.gr) = start(peak.gr) + 500
    end(peak.gr) = end(peak.gr) - 500
    return(peak.gr)
}

chromList = paste0("chr", 1:22)
inPath = "/shared/ngs/illumina/sfiorenz/201014_D00300_1082_AHHYJJBCX3/analysis"
index = grep("IP", list.files(inPath), fixed = TRUE)[!grep("IP", list.files(inPath), fixed = TRUE) %in% grep("IgG", list.files(inPath), fixed = TRUE)]
sampleL = list.files(inPath)[index]
sampleLtmp = sampleL %>% strsplit("_") %>% unlist
patientL = sampleLtmp[seq(4, length(sampleLtmp), 7)]
histL = sampleLtmp[seq(6, length(sampleLtmp), 7)]

peakAll = vector("list", 2)
peakAll[["H3K27me3"]] = vector("list", 30)
peakAll[["H3K4me2"]] = vector("list", 30)
mPeak = vector("list", 2)
mPeakEach = vector("list", 2)
mPeakAnno = vector("list", 2)

peakN = c()
peakLen = c()
peakCov = c()
peakOverlap = c()

topN <- 20
mergeType <- "Intersect"

In [50]:
for(i in 1:length(sampleL)){

    seacrControl = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_control.peaks.stringent.bed")) %>% filter(V1 %in% chromList, V5 >= 10, V4 > 2500)
    seacrTop = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_2.peaks.stringent.bed")) %>% filter(V1 %in% chromList, V5 >= 10, V4 > 2500) ##fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_01.peaks.stringent.bed")) %>% filter(V1 %in% chromList)
    seacrControl.gr = convertGR_extend(seacrControl)
    seacrTop.gr = convertGR_extend(seacrTop)

    seacrCT = findOverlaps(seacrControl.gr, seacrTop.gr)
    peak.gr = reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to])) ## reduce(append(seacrControl.gr, seacrTop.gr)) ## reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to]))
    peak.df = peak.gr %>% data.frame

    peakAll[[histL[i]]][[patientL[i]]] = peak.gr
    mPeakEach[[histL[i]]] = append(mPeakEach[[histL[i]]], peak.gr)
    
    peakN = data.frame(peakN = c(seacrControl %>% nrow, seacrTop %>% nrow, peak.df %>% nrow), type = c("SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakN, .)

    peakLen = data.frame(peakLen = c(seacrControl$V3 - seacrControl$V2, seacrTop$V3 - seacrTop$V2, peak.df$end - peak.df$start), type = c(rep("SEACR_Control", nrow(seacrControl)), rep(paste0("SEACR_Top", topN, "%"), nrow(seacrTop)), rep(mergeType, nrow(peak.df))), patient = patientL[i], histone = histL[i]) %>% rbind(peakLen, .)

    peakCov = data.frame(peakLen = c(sum(seacrControl$V3 - seacrControl$V2), sum(seacrTop$V3 - seacrTop$V2), sum(peak.df$end - peak.df$start)), type = c( "SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakCov, .)


    peakOverlap = data.frame(overlapN = c(seacrCT@from %>% unique %>% length, seacrCT@to %>% unique %>% length),
                             overlapProp =  c(seacrCT@from %>% unique %>% length/length(seacrControl.gr) * 100, seacrCT@to %>% unique %>% length/length(seacrTop.gr) *100), propLabel = c("SEACR_ControlvsTop/Control", "SEACR_ControlvsTop/Top"), numLabel = c( "SEACR_ControlvsTop:Control", "SEACR_ControlvsTop:Top"), patient = patientL[i], histone = histL[i]) %>% rbind(peakOverlap, .)

}

In [ ]:

outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/RData"
library(doMC)
coreN <- 6
registerDoMC(cores = coreN)

options(ChIPseeker.ignore_1st_exon = TRUE)
options(ChIPseeker.ignore_1st_intron = TRUE)
options(ChIPseeker.ignore_downstream = TRUE)
options(ChIPseeker.ignore_promoter_subcategory = TRUE)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
annodb <- "org.Hs.eg.db"

convertGR = function(peak){
    peak.gr = GRanges(seqname = peak$V1, IRanges(peak$V2, peak$V3), strand = "*")
    return(peak.gr)
}

chromList = paste0("chr", 1:22)
inPath = "/shared/ngs/illumina/sfiorenz/201014_D00300_1082_AHHYJJBCX3/analysis"
index = grep("IP", list.files(inPath), fixed = TRUE)[!grep("IP", list.files(inPath), fixed = TRUE) %in% grep("IgG", list.files(inPath), fixed = TRUE)]
sampleL = list.files(inPath)[index]
sampleLtmp = sampleL %>% strsplit("_") %>% unlist
patientL = sampleLtmp[seq(4, length(sampleLtmp), 7)]
histL = sampleLtmp[seq(6, length(sampleLtmp), 7)]

peakAll = vector("list", 2)
peakAll[["H3K27me3"]] = vector("list", 30)
peakAll[["H3K4me2"]] = vector("list", 30)
mPeak = vector("list", 2)
mPeakEach = vector("list", 2)
mPeakAnno = vector("list", 2)

peakN = c()
peakLen = c()
peakCov = c()
peakOverlap = c()

topN <- 20
mergeType <- "Intersect"
for(i in 1:length(sampleL)){

    seacrControl = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_control.peaks.stringent.bed")) %>% filter(V1 %in% chromList)
    seacrTop = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_2.peaks.stringent.bed")) %>% filter(V1 %in% chromList) ##fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_01.peaks.stringent.bed")) %>% filter(V1 %in% chromList)
    seacrControl.gr = convertGR(seacrControl)
    seacrTop.gr = convertGR(seacrTop)

    seacrCT = findOverlaps(seacrControl.gr, seacrTop.gr)
    peak.gr = reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to])) ## reduce(append(seacrControl.gr, seacrTop.gr)) ## reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to]))
    peak.df = peak.gr %>% data.frame

    peakAll[[histL[i]]][[patientL[i]]] = peak.gr
    mPeakEach[[histL[i]]] = append(mPeakEach[[histL[i]]], peak.gr)
    
    peakN = data.frame(peakN = c(seacrControl %>% nrow, seacrTop %>% nrow, peak.df %>% nrow), type = c("SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakN, .)

    peakLen = data.frame(peakLen = c(seacrControl$V3 - seacrControl$V2, seacrTop$V3 - seacrTop$V2, peak.df$end - peak.df$start), type = c(rep("SEACR_Control", nrow(seacrControl)), rep(paste0("SEACR_Top", topN, "%"), nrow(seacrTop)), rep(mergeType, nrow(peak.df))), patient = patientL[i], histone = histL[i]) %>% rbind(peakLen, .)

    peakCov = data.frame(peakLen = c(sum(seacrControl$V3 - seacrControl$V2), sum(seacrTop$V3 - seacrTop$V2), sum(peak.df$end - peak.df$start)), type = c( "SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakCov, .)


    peakOverlap = data.frame(overlapN = c(seacrCT@from %>% unique %>% length, seacrCT@to %>% unique %>% length),
                             overlapProp =  c(seacrCT@from %>% unique %>% length/length(seacrControl.gr) * 100, seacrCT@to %>% unique %>% length/length(seacrTop.gr) *100), propLabel = c("SEACR_ControlvsTop/Control", "SEACR_ControlvsTop/Top"), numLabel = c( "SEACR_ControlvsTop:Control", "SEACR_ControlvsTop:Top"), patient = patientL[i], histone = histL[i]) %>% rbind(peakOverlap, .)

}

inPath = "/shared/ngs/illumina/tphi/200924_D00300_1063_AHHLGCBCX3/analysis"
index = grep("IP", list.files(inPath), fixed = TRUE)[!grep("IP", list.files(inPath), fixed = TRUE) %in% grep("IgG", list.files(inPath), fixed = TRUE)]
sampleL = list.files(inPath)[index]
sampleLtmp = sampleL %>% strsplit("_") %>% unlist
patientL = sampleLtmp[seq(3, length(sampleLtmp), 6)]
histL = sampleLtmp[seq(5, length(sampleLtmp), 6)]

for(i in 1:length(sampleL)){

    seacrControl = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_control.peaks.stringent.bed")) %>% filter(V1 %in% chromList)
    seacrTop = fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_2.peaks.stringent.bed")) %>% filter(V1 %in% chromList) ## fread(paste0(inPath, "/", sampleL[i], "/seacr_norm_stringent_top0_01.peaks.stringent.bed")) %>% filter(V1 %in% chromList)
    seacrControl.gr = convertGR(seacrControl)
    seacrTop.gr = convertGR(seacrTop)

    seacrCT = findOverlaps(seacrControl.gr, seacrTop.gr)
    peak.gr = reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to])) ## reduce(append(seacrControl.gr, seacrTop.gr)) ##reduce(append(seacrControl.gr[seacrCT@from], seacrTop.gr[seacrCT@to]))
    peak.df = peak.gr %>% data.frame

    peakAll[[histL[i]]][[patientL[i]]] = peak.gr
    mPeakEach[[histL[i]]] = append(mPeakEach[[histL[i]]], peak.gr)
    
    peakN = data.frame(peakN = c(seacrControl %>% nrow, seacrTop %>% nrow, peak.df %>% nrow), type = c("SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakN, .)

    peakLen = data.frame(peakLen = c(seacrControl$V3 - seacrControl$V2, seacrTop$V3 - seacrTop$V2, peak.df$end - peak.df$start), type = c(rep("SEACR_Control", nrow(seacrControl)), rep(paste0("SEACR_Top", topN, "%"), nrow(seacrTop)), rep(mergeType, nrow(peak.df))), patient = patientL[i], histone = histL[i]) %>% rbind(peakLen, .)

    peakCov = data.frame(peakLen = c(sum(seacrControl$V3 - seacrControl$V2), sum(seacrTop$V3 - seacrTop$V2), sum(peak.df$end - peak.df$start)), type = c( "SEACR_Control", paste0("SEACR_Top", topN, "%"), mergeType), patient = patientL[i], histone = histL[i]) %>% rbind(peakCov, .)


    peakOverlap = data.frame(overlapN = c(seacrCT@from %>% unique %>% length, seacrCT@to %>% unique %>% length),
                             overlapProp =  c(seacrCT@from %>% unique %>% length/length(seacrControl.gr) * 100, seacrCT@to %>% unique %>% length/length(seacrTop.gr) *100), propLabel = c("SEACR_ControlvsTop/Control", "SEACR_ControlvsTop/Top"), numLabel = c( "SEACR_ControlvsTop:Control", "SEACR_ControlvsTop:Top"), patient = patientL[i], histone = histL[i]) %>% rbind(peakOverlap, .)

}


mPeak[["H3K27me3"]] = reduce(mPeakEach[["H3K27me3"]])
mPeak[["H3K4me2"]] = reduce(mPeakEach[["H3K4me2"]])

mPeakAnno[["H3K27me3"]] = annotatePeak(mPeak[["H3K27me3"]], tssRegion = c(-2000, 0), addFlankGeneInfo = TRUE,  TxDb = txdb, annoDb = annodb,  level = "gene")
mPeakAnno[["H3K4me2"]] = annotatePeak(mPeak[["H3K4me2"]], tssRegion = c(-2000, 0), addFlankGeneInfo = TRUE,  TxDb = txdb, annoDb = annodb,  level = "gene")

print(paste0("Total number of individual peaks: ", length(mPeakEach[["H3K27me3"]])))
print(paste0("Total number of merged master peaks: ", length(mPeak[["H3K27me3"]])))
print(paste0("Total number of individual peaks: ", length(mPeakEach[["H3K4me2"]])))
print(paste0("Total number of merged master peaks: ", length(mPeak[["H3K4me2"]])))

  

save(mPeak, peakAll, mPeakEach, mPeakAnno, peakN, peakCov, peakLen, peakOverlap, file = paste0(outPath, "/PatientData_peakOverlap_withDuplicates_seacrOnly_top0_", topN, ".RData"))
## save(mPeak, peakN, peakCov, peakLen, peakOverlap, file = paste0(outPath, "/PatientData_peakOverlap_withDuplicates_seacrOnly_top0_", topN, ".relaxed.union.RData"))

